In [20]:
from __future__ import print_function
import os
import argparse
import torch, gc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time  
import numpy as np
import pandas as pd

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from models.resnet import *
from models.vgg import *
from strategies.Dtrades import *

In [21]:
parser = argparse.ArgumentParser(description='PyTorch MNIST D-TRADES Adversarial Training')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=128, metavar='N',
                    help='input batch size for testing (default: 128)')
parser.add_argument('--epochs', type=int, default=50, metavar='N',#default=100
                    help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--epsilon', default=0.3,
                    help='perturbation')
parser.add_argument('--num-steps', default=20,
                    help='perturb number of steps')
parser.add_argument('--step-size', default=0.01,
                    help='perturb step size')
parser.add_argument('--alpha', default=1,
                    help='regularization, i.e., 1/lambda in D-TRADES')
parser.add_argument('--beta', default=1,
                    help='regularization, i.e., 1/lambda in D-TRADES')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--model-dir', default='./testing',
                    help='directory of model for saving checkpoint')
parser.add_argument('--save-freq', '-s', default=1, type=int, metavar='N',
                    help='save frequency')
parser.add_argument('--model', type=str, default='resnet', choices=['resnet', 'vgg'],
                    help='Modelo a usar: resnet o vgg')
args, unknown = parser.parse_known_args()


In [22]:
# Ajustes
model_dir = args.model_dir
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [23]:
# Configurar el cargador de datos
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.ToTensor()),
                   batch_size=args.test_batch_size, shuffle=False, **kwargs)

In [24]:
lambda_min = []
lambda_max = []
lambda_mean = []
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss, lambda_value = d_trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=args.step_size,
                           epsilon=args.epsilon,
                           perturb_steps=args.num_steps,
                           alpha=args.alpha,
                           beta=args.beta,
                        )
        loss.backward()
        optimizer.step()

        # print progress
        if batch_idx % args.log_interval == 0:
            lambda_min.append(lambda_value.min().item())
            lambda_max.append(lambda_value.max().item())
            lambda_mean.append(lambda_value.mean().item())
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tLambda Mean: {:.2f} \tLambda Min: {:.2f} \tLambda Max: {:.2f}'.format(
                                            epoch,
                                            batch_idx * len(data),
                                            len(train_loader.dataset),
                                            100. * batch_idx / len(train_loader),
                                            loss.item(),
                                            lambda_value.mean().item(),
                                            lambda_value.min().item(),
                                            lambda_value.max().item()))

In [25]:
def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args.epsilon,
                  num_steps=20,
                  step_size=0.003):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)

    random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
    X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    return err, err_pgd

In [26]:
def eval_adv_test_whitebox(model, device, test_loader):

    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
        
    natural_acc = 1 - natural_err_total / len(test_loader.dataset)
    robust_acc = 1- robust_err_total / len(test_loader.dataset)
    robust_drop = natural_acc - robust_acc
    attack_success_rate = 1 - robust_acc
    
    print(f'PGD natural_acc: {natural_acc:.4f}, robust_acc: {robust_acc:.4f}, robust_drop: {robust_drop:4f}, attack_success_rate: {attack_success_rate:4f}')
    return natural_acc, robust_acc, robust_drop, attack_success_rate

In [27]:
def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate"""
    lr = args.lr
    if epoch >= 55:
        lr = args.lr * 0.1
    if epoch >= 75:
        lr = args.lr * 0.01
    if epoch >= 90:
        lr = args.lr * 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [28]:
gc.collect()
torch.cuda.empty_cache()

In [29]:
def main():
    if args.model.lower() == "resnet":
        model = ResNet18()
        model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
    elif args.model.lower() == "vgg":
        model = vgg16(in_channels=1)
    else:
        raise ValueError("Modelo no reconocido: usa --model resnet o --model vgg")

    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    # Historial
    history = {'natural_acc': [], 'robust_acc': [], 'robust_drop': [], 'attack_succes_rate': []}
    results = {}
    
    for epoch in range(1, args.epochs + 1):
        adjust_learning_rate(optimizer, epoch)
        
        start_time = time.time()

        # Entrenamiento adversarial
        train(args, model, device, train_loader, optimizer, epoch)

        print('================================================================')
        
        # Evaluación
        natural_acc, robust_acc, robust_drop, attack_succes_rate = eval_adv_test_whitebox(
            model, device, test_loader
        )

        print('using time:', time.time() - start_time)
        print('================================================================')
        
        # Guardar historial
        history['natural_acc'].append(natural_acc)
        history['robust_acc'].append(robust_acc)
        history['robust_drop'].append(robust_drop)
        history['attack_succes_rate'].append(attack_succes_rate)

        results[f'history{epoch}'] = history.copy()

        # Guardar checkpoints
        if epoch % args.save_freq == 0:
            torch.save(model.state_dict(),
                       os.path.join(model_dir, f'model-nn-epoch{epoch}.pt'))
            torch.save(optimizer.state_dict(),
                       os.path.join(model_dir, f'opt-nn-checkpoint_epoch{epoch}.tar'))
    
    print('================================================================')
    print("\nResumen de Resultados por historial:")
    for ep, hist in results.items():
        print(f"{ep}: Max Robust Acc: {max(hist['robust_acc']):.4f}")

if __name__ == '__main__':
    main()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.401673 	Lambda Mean: 1.05 	Lambda Min: 0.81 	Lambda Max: 1.22
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.266848 	Lambda Mean: 0.34 	Lambda Min: 0.00 	Lambda Max: 1.47
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.221973 	Lambda Mean: 0.31 	Lambda Min: 0.00 	Lambda Max: 1.41


KeyboardInterrupt: 